In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200907


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_退会顧客")

In [6]:
prefcode_list = "code-todoufuken"
prefcode_list = pd.read_excel(data_dir /r"Input"/f"{prefcode_list}.xlsx",encoding='shift-jis')
prefcode_list.head()

,prefecture,prefecture2
0,北海道,01_北海道
1,青森県,02_青森県
2,岩手県,03_岩手県
3,宮城県,04_宮城県
4,秋田県,05_秋田県


前々月までの実績をSASで、最新実績を速報でCognosで

まずは当月解約した顧客の取込

In [7]:
df_new = pd.read_csv(data_dir /r"Input"/"CANCEL_CUST_SHOPPING_cognos.csv",encoding = 'shift-jis',dtype={'serialno':str})
df_new.head()

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg
0,155206141642,20200728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,155208624736,20200826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,155211025889,20200804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,155211346574,20200801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155211715380,20200820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_new = df_new.query('kzk_f == 0 and cancel_date >= 20200701')
df_new.head()

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg
61,155202803245,20200816,20200731.0,4980006001111007,ゴールド,Visa,郵送,0.0,0.0,19371210.0,80歳以上,男性,1000.0,11_1000万円以上,奈良県,0.0,0.0,A,0.0,0.0,20070514.0,10年以上,0.0,0.0,7.0,200707.0,0.0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,155205852769,20200725,20200731.0,4980006004327006,ゴールド,Visa,郵送,20190616.0,0.0,19350226.0,80歳以上,男性,550.0,06_600万円未満,京都府,9.0,0.0,A,1.0,0.0,20070518.0,10年以上,0.0,0.0,72.0,200707.0,20200205.0,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,155206531222,20200705,20200731.0,4980006004596006,ゴールド,Visa,郵送,20070914.0,0.0,19390816.0,80歳以上,男性,700.0,08_800万円未満,京都府,1.0,0.0,6,0.0,0.0,20070518.0,10年以上,0.0,0.0,0.0,200707.0,20100607.0,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,155206537336,20200826,20200731.0,4980006004608108,ゴールド,Visa,郵送,20200328.0,0.0,19290312.0,80歳以上,男性,0.0,00_0円,京都府,0.0,0.0,9,0.0,0.0,20070518.0,10年以上,0.0,13.0,13.0,200707.0,20150703.0,Ｄ_京都店,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13156.0,0.0,13156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,JFR-G
65,155202673614,20200729,20200731.0,4980006010515008,ゴールド,Visa,郵送,20141208.0,0.0,19510303.0,65〜69歳,男性,0.0,00_0円,兵庫県,3.0,0.0,9,1.0,0.0,20070529.0,10年以上,0.0,0.0,10.0,200708.0,0.0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-G


In [9]:
print(df_new.dtypes)

daihyouno                 int64
cancel_date               int64
cp_date                 float64
serialno                 object
card                     object
brand                    object
channel                  object
last_use_date           float64
kzk_f                   float64
birthday                float64
age                      object
gender                   object
income                  float64
income2                  object
prefecture               object
job                     float64
occupation              float64
residense                object
email_f                 float64
web_f                   float64
kanyu_date              float64
membership_term          object
shopping_amt_3month     float64
shopping_amt_6month     float64
shopping_amt_12month    float64
nenkaihi                float64
kirikae_date            float64
dm_code                  object
credit_use_months       float64
department_amt          float64
g_oversea_amt           float64
g_mailor

In [10]:
df_new = df_new.astype({'cp_date':int,'birthday':int,'kanyu_date':int,'last_use_date':int,'nenkaihi':int,'kirikae_date':int})
df_new.head()

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg
61,155202803245,20200816,20200731,4980006001111007,ゴールド,Visa,郵送,0,0.0,19371210,80歳以上,男性,1000.0,11_1000万円以上,奈良県,0.0,0.0,A,0.0,0.0,20070514,10年以上,0.0,0.0,7.0,200707,0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,155205852769,20200725,20200731,4980006004327006,ゴールド,Visa,郵送,20190616,0.0,19350226,80歳以上,男性,550.0,06_600万円未満,京都府,9.0,0.0,A,1.0,0.0,20070518,10年以上,0.0,0.0,72.0,200707,20200205,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,155206531222,20200705,20200731,4980006004596006,ゴールド,Visa,郵送,20070914,0.0,19390816,80歳以上,男性,700.0,08_800万円未満,京都府,1.0,0.0,6,0.0,0.0,20070518,10年以上,0.0,0.0,0.0,200707,20100607,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,155206537336,20200826,20200731,4980006004608108,ゴールド,Visa,郵送,20200328,0.0,19290312,80歳以上,男性,0.0,00_0円,京都府,0.0,0.0,9,0.0,0.0,20070518,10年以上,0.0,13.0,13.0,200707,20150703,Ｄ_京都店,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13156.0,0.0,13156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,JFR-G
65,155202673614,20200729,20200731,4980006010515008,ゴールド,Visa,郵送,20141208,0.0,19510303,65〜69歳,男性,0.0,00_0円,兵庫県,3.0,0.0,9,1.0,0.0,20070529,10年以上,0.0,0.0,10.0,200708,0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-G


In [11]:
#↑もらったCognos抽出リストには、臨時や家族カードなどが混じっていたため除外する
df_exc = pd.read_csv(data_dir /r"Input"/"7月8月解約客_from_cognos.csv",encoding = 'shift-jis')
df_exc.head()

,daihyouno
0,155232276826
1,155224308512
2,155214888291
3,155220363164
4,155225268509


In [12]:
#マージし除外
df_new = pd.merge(df_new,df_exc,on=['daihyouno'],how = 'inner')
df_new.shape[0]

24974

In [13]:
df_new['daihyouno'].nunique()

23370

In [14]:
#重複削除
df_new = df_new.drop_duplicates(subset=['daihyouno'])

In [15]:
df_new.shape[0]

23370

In [16]:
_df_new = df_new.copy()

In [17]:
df_new['cancel_date']=pd.to_datetime(df_new['cancel_date'], format='%Y%m%d')

In [18]:
df_new['cancel_year'] = df_new['cancel_date'].dt.year
df_new['cancel_month'] = df_new['cancel_date'].dt.month
df_new['cancel_day'] = df_new['cancel_date'].dt.day

In [19]:
#日付データ文字化
df_new['cancel_date'] = df_new['cancel_date'].dt.strftime('%Y/%m/%d')

In [44]:
#解約日次推移
q = """
        SELECT
        cancel_date
        ,cancel_year
        ,cancel_month
        ,cancel_day
        ,card
        ,count(daihyouno) as sum_no_of_account
        from 
            df_new
        group by
            1,2,3,4,5
        """
df_ts = sqldf(q, locals())
df_ts.head()

,cancel_date,cancel_year,cancel_month,cancel_day,card,sum_no_of_account
0,2020/07/01,2020,7,1,ゴールド,22
1,2020/07/01,2020,7,1,ハウスゴールド,22
2,2020/07/01,2020,7,1,一般,767
3,2020/07/01,2020,7,1,外商,41
4,2020/07/02,2020,7,2,ゴールド,5


In [18]:
#CSV出力
df_ts.to_csv(data_dir /'解約数日次推移_cognos.csv', header=True ,index=None, encoding='shift-jis')

In [45]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23370 entries, 0 to 24973
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   daihyouno             23370 non-null  int64  
 1   cancel_date           23370 non-null  object 
 2   cp_date               23370 non-null  int32  
 3   serialno              23370 non-null  object 
 4   card                  23370 non-null  object 
 5   brand                 23370 non-null  object 
 6   channel               23370 non-null  object 
 7   last_use_date         23370 non-null  int32  
 8   kzk_f                 23370 non-null  float64
 9   birthday              23370 non-null  int32  
 10  age                   23370 non-null  object 
 11  gender                23370 non-null  object 
 12  income                23370 non-null  float64
 13  income2               23370 non-null  object 
 14  prefecture            23370 non-null  object 
 15  job                

過去の退会数データを取込

In [305]:
df_exist = pd.read_csv(data_dir /r"Input"/"CANCEL_CUST_SHOPPING.csv",encoding = 'shift-jis',dtype={'serialno':str})

In [306]:
df_exist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192126 entries, 0 to 192125
Data columns (total 52 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cp_date               192126 non-null  int64  
 1   daihyouno             192126 non-null  int64  
 2   serialno              192126 non-null  object 
 3   card                  192126 non-null  object 
 4   brand                 192126 non-null  object 
 5   channel               192126 non-null  object 
 6   cancel_date           192126 non-null  int64  
 7   cancel_yymm           192126 non-null  int64  
 8   last_use_date         192126 non-null  int64  
 9   kzk_f                 192126 non-null  int64  
 10  birthday              192126 non-null  int64  
 11  age                   192126 non-null  object 
 12  gender                192126 non-null  object 
 13  income                192126 non-null  int64  
 14  income2               192126 non-null  object 
 15  

In [307]:
print(df_exist['cancel_yymm'].unique())

[202002 201905 201909 201912 202004 202006 201903 201906 201911 201910
 201904 202005 202007 201907 201908 202001 202003 202008]


In [308]:
#当月分が混ざっているので除外
df_exist = df_exist.query('cancel_yymm < 202008')
df_exist

,cp_date,daihyouno,serialno,card,brand,channel,cancel_date,cancel_yymm,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt
0,20200731,155202803245,4980006001111007,ゴールド,Visa,郵送,20200215,202002,0,0,19371210,80歳以上,男性,1000,11_1000万円以上,奈良県,0,0,A,0,0,A,20070514,10年以上,0,0,7,200707,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20200731,155205651583,4980006001308009,ゴールド,Visa,郵送,20190523,201905,20181226,0,19361102,80歳以上,男性,0,00_0円,京都府,1,0,9,0,0,A,20070514,10年以上,0,0,0,200707,0,1.0,3500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200731,155201966217,4980006003030007,ゴールド,Visa,郵送,20190909,201909,0,0,19440130,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,B,20070518,10年以上,0,0,0,200707,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20200731,155202465250,4980006003423004,ゴールド,Visa,郵送,20191202,201912,20191127,0,19590718,60〜64歳,男性,1750,11_1000万円以上,大阪府,0,0,0,0,0,A,20070518,10年以上,0,0,2,200707,0,1.0,2200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200731,155202816262,4980006003768002,ゴールド,Visa,郵送,20200408,202004,20191128,0,19450603,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,A,20070518,10年以上,0,0,9,200707,0,1.0,8788.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192121,20200731,155227900810,5279262703813005,外商,Visa,郵送,20190628,201906,20190815,0,19560813,60〜64歳,男性,732,08_800万円未満,広島県,0,91,0,1,0,P,20151204,4年〜5年未満,0,0,0,201601,0,6.0,68496.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,792527.0,0.0,792527.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,156.0,0.0,156.0
192122,20200731,155228077766,5279262703952001,外商,Visa,郵送,20200307,202003,20191011,0,19420925,75〜79歳,女性,500,06_600万円未満,高知県,9,93,9,0,0,A,20160121,4年〜5年未満,0,0,21,201603,0,5.0,118421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192123,20200731,155228467017,5279262704191203,外商,Visa,郵送,20190629,201906,20190828,0,19491022,70〜74歳,女性,1800,11_1000万円以上,高知県,0,60,0,1,0,P,20160428,4年〜5年未満,0,0,-554,201606,20180627,12.0,28606.0,2295101.0,0.0,0.0,0.0,0.0,0.0,0.0,883160.0,0.0,3178261.0,4.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,175.0,0.0,218.0
192124,20200731,155229509767,5279262704874006,外商,Visa,郵送,20190618,201906,20180908,0,19540114,65〜69歳,男性,780,08_800万円未満,高知県,0,91,0,0,0,P,20170112,3年〜4年未満,0,0,0,201702,0,2.0,44575.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
#日付データ変換前の一時対比したdf_newのコピーを呼び込み、縦結合
_df_new = _df_new.query('kzk_f == 0 and cancel_date >= 20200801')
df = pd.concat([_df_new,df_exist])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201848 entries, 0 to 192125
Data columns (total 54 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   daihyouno             201848 non-null  int64  
 1   cancel_date           201848 non-null  int64  
 2   cp_date               201848 non-null  int64  
 3   serialno              201848 non-null  object 
 4   card                  201848 non-null  object 
 5   brand                 201848 non-null  object 
 6   channel               201848 non-null  object 
 7   last_use_date         201848 non-null  int64  
 8   kzk_f                 201848 non-null  float64
 9   birthday              201848 non-null  int64  
 10  age                   201848 non-null  object 
 11  gender                201848 non-null  object 
 12  income                201848 non-null  float64
 13  income2               201848 non-null  object 
 14  prefecture            201848 non-null  object 
 15  

In [310]:
df['cp_date']=pd.to_datetime(df['cp_date'], format='%Y%m%d')#基準月
df['cancel_date']=pd.to_datetime(df['cancel_date'], format='%Y%m%d')
df['kanyu_date']=pd.to_datetime(df['kanyu_date'], format='%Y%m%d')#加入日
df['nenkaihi'] = pd.to_datetime(df['nenkaihi'], format="%Y%m") + MonthEnd(1)#年会費
df.head()
#どうにもlast_use_dateを変換できないので、今はわからないのであきらめる

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg,cancel_yymm,cancel_code
0,155202803245,2020-08-16,2020-07-31,4980006001111007,ゴールド,Visa,郵送,0,0.0,19371210,80歳以上,男性,1000.0,11_1000万円以上,奈良県,0.0,0.0,A,0.0,0.0,2007-05-14,10年以上,0.0,0.0,7.0,2007-07-31,0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,155206537336,2020-08-26,2020-07-31,4980006004608108,ゴールド,Visa,郵送,20200328,0.0,19290312,80歳以上,男性,0.0,00_0円,京都府,0.0,0.0,9,0.0,0.0,2007-05-18,10年以上,0.0,13.0,13.0,2007-07-31,20150703,Ｄ_京都店,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13156.0,0.0,13156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,JFR-G,NaN,NaN
6,155209679481,2020-08-28,2020-07-31,4980006012706001,ゴールド,Visa,郵送,20070819,0.0,19630709,55〜59歳,女性,400.0,05_500万円未満,兵庫県,1.0,0.0,0,0.0,0.0,2007-05-29,10年以上,0.0,0.0,0.0,2007-08-31,20070814,Ｄ_神戸店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,155206832950,2020-08-06,2020-07-31,4980006017765002,ゴールド,Visa,郵送,20121127,0.0,19330211,80歳以上,男性,0.0,00_0円,京都府,3.0,0.0,9,0.0,0.0,2007-06-18,10年以上,0.0,0.0,0.0,2007-08-31,0,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-G,NaN,NaN
20,155202496222,2020-08-05,2020-07-31,4980006020372101,ゴールド,Visa,郵送,20180128,0.0,19310313,80歳以上,男性,1500.0,11_1000万円以上,大阪府,0.0,0.0,0,0.0,0.0,2007-06-21,10年以上,0.0,0.0,0.0,2007-08-31,20101124,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-G,NaN,NaN


In [311]:
df['cancel_year'] = df['cancel_date'].dt.year
df['cancel_month'] = df['cancel_date'].dt.month

df['cp_date'] = df['cp_date'].dt.strftime('%Y/%m/%d')
df['cancel_date'] = df['cancel_date'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_date'].dt.strftime('%Y/%m/%d')
df['nenkaihi'] = df['nenkaihi'].dt.strftime('%Y/%m/%d')

In [312]:
df['age'] = df['age'].str.replace('〜','-')

In [313]:
print(df['age'].unique())

['80歳以上' '55-59歳' '60-64歳' '70-74歳' '40-44歳' '45-49歳' '65-69歳' '35-39歳'
 '50-54歳' '30-34歳' '75-79歳' '25-29歳' '20-24歳' '19歳以下（20歳未満）']


#日付無しデータをダミーの未来日付設定
for i in range(len(df)):
    if df['last_use_date'][i] == 0:
        df.loc[:,'last_use_date'][i] = 20501231#最終利用日

In [ ]:
#どういった会員が退会したかデモグラ

In [314]:
#性・年代別
q = """
        SELECT
        cancel_year
        ,cancel_month
        ,card
        ,gender
        ,case 
        when age like '3%' then '30代'
        when age like '4%' then '40代'
        when age like '5%' then '50代'
        when age like '6%' then '60代'
        when age like '7%' then '70代'
        when age like '8%' then '80代'
        else '20代'
        end as age_range
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4,5
        """
df_age = sqldf(q, locals())
df_age

,cancel_year,cancel_month,card,gender,age_range,sum_no_of_account
0,2019,3,ゴールド,女性,20代,3
1,2019,3,ゴールド,女性,30代,18
2,2019,3,ゴールド,女性,40代,27
3,2019,3,ゴールド,女性,50代,26
4,2019,3,ゴールド,女性,60代,25
...,...,...,...,...,...,...
737,2020,8,外商,男性,40代,23
738,2020,8,外商,男性,50代,46
739,2020,8,外商,男性,60代,38
740,2020,8,外商,男性,70代,47


In [315]:
#CSV出力
df_age.to_csv(data_dir /'cancel_by_age.csv', header=True ,index=None, encoding='shift-jis')

In [316]:
#年収別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,income2
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_income = sqldf(q, locals())
df_income.head()

,card,cancel_year,cancel_month,income2,sum_no_of_account
0,ゴールド,2019,3,00_0円,23
1,ゴールド,2019,3,01_100万円未満,2
2,ゴールド,2019,3,02_200万円未満,11
3,ゴールド,2019,3,03_300万円未満,12
4,ゴールド,2019,3,04_400万円未満,24


In [317]:
#CSV出力
df_income.to_csv(data_dir /'cancel_by_income.csv', header=True ,index=None, encoding='shift-jis')

In [318]:
df = pd.merge(df,prefcode_list, on =['prefecture'], how = 'left')

In [319]:
#エリア別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,prefecture2
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_pref = sqldf(q, locals())
df_pref

,card,cancel_year,cancel_month,prefecture2,sum_no_of_account
0,ゴールド,2019,3,01_北海道,22
1,ゴールド,2019,3,04_宮城県,1
2,ゴールド,2019,3,08_茨城県,4
3,ゴールド,2019,3,10_群馬県,1
4,ゴールド,2019,3,11_埼玉県,1
...,...,...,...,...,...
1701,外商,2020,8,35_山口県,16
1702,外商,2020,8,37_香川県,1
1703,外商,2020,8,38_愛媛県,1
1704,外商,2020,8,39_高知県,19


In [320]:
#CSV出力
df_pref.to_csv(data_dir /'cancel_by_pref.csv', header=True ,index=None, encoding='shift-jis')

In [321]:
#職業別
q = """
        select
        card
        ,cancel_year
        ,cancel_month
        ,job
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_job = sqldf(q, locals())
df_job

,card,cancel_year,cancel_month,job,sum_no_of_account
0,ゴールド,2019,3,0.0,161
1,ゴールド,2019,3,1.0,2
2,ゴールド,2019,3,3.0,15
3,ゴールド,2019,3,4.0,2
4,ゴールド,2019,3,5.0,8
...,...,...,...,...,...
263,外商,2020,6,9.0,117
264,外商,2020,7,0.0,416
265,外商,2020,7,9.0,158
266,外商,2020,8,0.0,268


In [322]:
#CSV出力
df_job.to_csv(data_dir /'cancel_by_job.csv', header=True ,index=None, encoding='shift-jis')

In [323]:
#加入年月別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,membership_term
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_term = sqldf(q, locals())
df_term

,card,cancel_year,cancel_month,membership_term,sum_no_of_account
0,ゴールド,2019,3,10年以上,16
1,ゴールド,2019,3,1年〜2年未満,11
2,ゴールド,2019,3,2年〜3年未満,51
3,ゴールド,2019,3,3年〜4年未満,37
4,ゴールド,2019,3,4年〜5年未満,35
...,...,...,...,...,...
531,外商,2020,8,3年〜4年未満,33
532,外商,2020,8,4年〜5年未満,59
533,外商,2020,8,5年〜6年未満,44
534,外商,2020,8,6ヶ月〜1年未満,20


In [324]:
#CSV出力
df_term.to_csv(data_dir /'cancel_by_term.csv', header=True ,index=None, encoding='shift-jis')

In [325]:
#居住状況別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,residense
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_resi = sqldf(q, locals())
df_resi

,card,cancel_year,cancel_month,residense,sum_no_of_account
0,ゴールド,2019,3,0,54
1,ゴールド,2019,3,1,24
2,ゴールド,2019,3,5,3
3,ゴールド,2019,3,6,38
4,ゴールド,2019,3,7,9
...,...,...,...,...,...
564,外商,2020,8,9,12
565,外商,2020,8,A,197
566,外商,2020,8,B,1
567,外商,2020,8,D,13


In [326]:
#CSV出力
df_resi.to_csv(data_dir /'cancel_by_residense.csv', header=True ,index=None, encoding='shift-jis')

In [327]:
#Web明細有無別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,web_f
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_web = sqldf(q, locals())
df_web

,card,cancel_year,cancel_month,web_f,sum_no_of_account
0,ゴールド,2019,3,0.0,175
1,ゴールド,2019,3,1.0,54
2,ゴールド,2019,4,0.0,148
3,ゴールド,2019,4,1.0,52
4,ゴールド,2019,5,0.0,166
...,...,...,...,...,...
105,外商,2020,6,1.0,37
106,外商,2020,7,0.0,536
107,外商,2020,7,1.0,38
108,外商,2020,8,0.0,334


In [328]:
#CSV出力
df_web.to_csv(data_dir /'cancel_by_web.csv', header=True ,index=None, encoding='shift-jis')

In [329]:
#入会チャネル別
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,channel
        ,count(daihyouno) as sum_no_of_account
        from 
            df
        group by
            1,2,3,4
        """
df_channel = sqldf(q, locals())
df_channel

,card,cancel_year,cancel_month,channel,sum_no_of_account
0,ゴールド,2019,3,オンライン,14
1,ゴールド,2019,3,店頭タブレット,70
2,ゴールド,2019,3,郵送,145
3,ゴールド,2019,4,オンライン,16
4,ゴールド,2019,4,店頭タブレット,68
...,...,...,...,...,...
122,外商,2020,4,郵送,503
123,外商,2020,5,郵送,406
124,外商,2020,6,郵送,471
125,外商,2020,7,郵送,574


In [330]:
#CSV出力
df_channel.to_csv(data_dir /'cancel_by_channel.csv', header=True ,index=None, encoding='shift-jis')

利用加盟店状況

q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,case 
            when credit_use_months <= 0 then 0 
            else credit_use_months 
            end as use_months
        ,count(daihyouno) as sum_no_of_account
        from 
            _df_demo
        group by
            1,2,3,4
        """
df_usemonth = sqldf(q, locals())
df_usemonth

In [331]:
df[['daihyouno','department_amt','gaibu_amt']].head(10)

,daihyouno,department_amt,gaibu_amt
0,155202803245,NaN,NaN
1,155206537336,0.0,13156.0
2,155209679481,NaN,NaN
3,155206832950,NaN,NaN
4,155202496222,NaN,NaN
5,155209866401,NaN,NaN
6,155211027299,NaN,NaN
7,155202156289,NaN,NaN
8,155211025871,52340.0,305269.0
9,155209075508,NaN,NaN


In [332]:
_df = df.copy()
#df = _df.copy

In [333]:
df['depart_amt_wt'] = ((df['department_amt']/(df['department_amt'] + df['gaibu_amt']))*100)//10 * 10

In [334]:
df[['daihyouno','department_amt','gaibu_amt','depart_amt_wt']].query('depart_amt_wt < 0')

,daihyouno,department_amt,gaibu_amt,depart_amt_wt
18106,155227670967,883.0,-1285.0,-220.0
35253,155203235041,271.0,-377800.0,-10.0
51480,155218927301,16681.0,-76152.0,-30.0
68194,155228508976,19747.0,-122963.0,-20.0
68720,155228671436,10356.0,-11200.0,-1230.0
70792,155229486008,-3888.0,20994.0,-30.0
83000,155232595647,583.0,-29963.0,-10.0
96344,155229146693,-540.0,2700.0,-30.0
99328,155229860749,-3614.0,16910.0,-30.0
131063,155226083055,28772.0,-217982.0,-20.0


#マイナスを0に置換
for i in range(len(df)):
    if np.isnan(df['department_amt'][i]) == True | df['department_amt'][i] < 0:
        df.loc[:,'ASID'][i] = 0

In [335]:
df['depart_amt_wt'] = ((df['department_amt']/(df['department_amt'] + df['gaibu_amt']))*100)//10 * 10
df[['daihyouno','department_amt','gaibu_amt','depart_amt_wt']].head(10)

,daihyouno,department_amt,gaibu_amt,depart_amt_wt
0,155202803245,NaN,NaN,NaN
1,155206537336,0.0,13156.0,0.0
2,155209679481,NaN,NaN,NaN
3,155206832950,NaN,NaN,NaN
4,155202496222,NaN,NaN,NaN
5,155209866401,NaN,NaN,NaN
6,155211027299,NaN,NaN,NaN
7,155202156289,NaN,NaN,NaN
8,155211025871,52340.0,305269.0,10.0
9,155209075508,NaN,NaN,NaN


In [336]:
df[['daihyouno','department_amt','gaibu_amt','depart_amt_wt']].query('depart_amt_wt >100')

,daihyouno,department_amt,gaibu_amt,depart_amt_wt
286,155227554252,3094.0,-984.0,140.0
364,155227520733,166018.0,-15640.0,110.0
60438,155218756916,16092.0,-3909.0,130.0
69362,155228929123,32118.0,-15572.0,190.0
75249,155230999098,10660.0,-1892.0,120.0
88541,155227163971,177422.0,-141600.0,490.0
95738,155228987865,20771.0,-2970.0,110.0
98731,155229718699,3528.0,-1566.0,170.0
104984,155230996201,68850.0,-39378.0,230.0
134029,155220025276,10049.0,-2204.0,120.0


In [337]:
df.groupby(['depart_amt_wt'])['daihyouno'].count()

depart_amt_wt
-2620.0        1
-1230.0        1
-220.0         1
-100.0         1
-90.0          1
-60.0          1
-50.0          1
-30.0          4
-20.0          2
-10.0          5
 0.0       28337
 10.0       3303
 20.0       2286
 30.0       1781
 40.0       1523
 50.0       1448
 60.0       1408
 70.0       1564
 80.0       1795
 90.0       2698
 100.0     35262
 110.0         3
 120.0         3
 130.0         1
 140.0         1
 170.0         1
 190.0         1
 210.0         1
 220.0         1
 230.0         1
 260.0         1
 490.0         1
Name: daihyouno, dtype: int64

In [338]:
#百貨店メインユーザー
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,case 
            when depart_amt_wt < 0 then 0
            when depart_amt_wt > 100 then 100
            else depart_amt_wt
            end as depart_amt_wt2
        ,count(daihyouno) as sum_no_of_account
        ,sum(department_amt) as sum_depart_amt
        ,sum(gaibu_amt) as sum_gaibu_amt
        from 
            df
        group by
            1,2,3,4
        """
df_wt = sqldf(q, locals())
df_wt

,card,cancel_year,cancel_month,depart_amt_wt2,sum_no_of_account,sum_depart_amt,sum_gaibu_amt
0,ゴールド,2019,3,NaN,57,0.0,0.0
1,ゴールド,2019,3,0.0,75,1497370.0,51577838.0
2,ゴールド,2019,3,10.0,13,1720880.0,8999171.0
3,ゴールド,2019,3,20.0,10,1115993.0,3372250.0
4,ゴールド,2019,3,30.0,9,1427214.0,2600039.0
...,...,...,...,...,...,...,...
644,外商,2020,8,60.0,7,2210920.0,1176944.0
645,外商,2020,8,70.0,5,1512286.0,573659.0
646,外商,2020,8,80.0,2,120366.0,25065.0
647,外商,2020,8,90.0,17,11832638.0,600850.0


In [339]:
#CSV出力
df_wt.to_csv(data_dir /'cancel_by_depart_amt_wt.csv', header=True ,index=None, encoding='shift-jis')

#CSV出力
df_usemonth.to_csv(data_dir /'解約_cognos_usemonth.csv', header=True ,index=None, encoding='shift-jis')

2020/9/7 Adhoc分析　当月解約掘り下げ（外商）

In [20]:
df_new

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg,cancel_year,cancel_month,cancel_day
0,155202803245,2020/08/16,20200731,4980006001111007,ゴールド,Visa,郵送,0,0.0,19371210,80歳以上,男性,1000.0,11_1000万円以上,奈良県,0.0,0.0,A,0.0,0.0,20070514,10年以上,0.0,0.0,7.0,200707,0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,8,16
1,155206531222,2020/07/05,20200731,4980006004596006,ゴールド,Visa,郵送,20070914,0.0,19390816,80歳以上,男性,700.0,08_800万円未満,京都府,1.0,0.0,6,0.0,0.0,20070518,10年以上,0.0,0.0,0.0,200707,20100607,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,7,5
3,155206537336,2020/08/26,20200731,4980006004608108,ゴールド,Visa,郵送,20200328,0.0,19290312,80歳以上,男性,0.0,00_0円,京都府,0.0,0.0,9,0.0,0.0,20070518,10年以上,0.0,13.0,13.0,200707,20150703,Ｄ_京都店,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13156.0,0.0,13156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,JFR-G,2020,8,26
4,155202673614,2020/07/29,20200731,4980006010515008,ゴールド,Visa,郵送,20141208,0.0,19510303,65〜69歳,男性,0.0,00_0円,兵庫県,3.0,0.0,9,1.0,0.0,20070529,10年以上,0.0,0.0,10.0,200708,0,Ｄ_心斎橋店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-G,2020,7,29
5,155205046552,2020/07/30,20200731,4980006010850207,ゴールド,Visa,郵送,20200119,0.0,19480623,70〜74歳,男性,1900.0,11_1000万円以上,東京都,1.0,0.0,9,0.0,0.0,20070529,10年以上,0.0,0.0,71.0,200708,20170605,Ｄ_東京店,3.0,48924.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21760.0,0.0,21760.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,JFR-G,2020,7,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24969,155209706623,2020/07/03,20200731,6900102605490004,ハウスゴールド,ハウスゴールド,郵送,20191014,0.0,19681212,50〜54歳,女性,0.0,00_0円,兵庫県,9.0,0.0,9,0.0,0.0,20080808,10年以上,0.0,0.0,0.0,200811,0,Ｄ_神戸店,1.0,493.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JFR-H,2020,7,3
24970,155210198596,2020/08/07,20200731,6900102613561002,ハウスゴールド,ハウスゴールド,郵送,20090314,0.0,19470925,70〜74歳,女性,0.0,00_0円,兵庫県,3.0,0.0,9,0.0,0.0,20080808,10年以上,0.0,0.0,0.0,200908,0,Ｄ_神戸店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-H,2020,8,7
24971,155211178035,2020/08/22,20200731,6900102616562007,ハウスゴールド,ハウスゴールド,郵送,0,0.0,19410408,75〜79歳,女性,0.0,00_0円,兵庫県,3.0,0.0,9,0.0,0.0,20080811,10年以上,0.0,0.0,0.0,200906,0,Ｄ_神戸店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-H,2020,8,22
24972,155210164655,2020/08/16,20200731,6900102617185105,ハウスゴールド,ハウスゴールド,郵送,0,0.0,19680417,50〜54歳,女性,0.0,00_0円,兵庫県,3.0,0.0,9,0.0,0.0,20080811,10年以上,0.0,0.0,0.0,200906,20200316,Ｄ_神戸店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-H,2020,8,16


In [22]:
df_new_gaisho = df_new.query('card == "外商"')
df_new_gaisho

,daihyouno,cancel_date,cp_date,serialno,card,brand,channel,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date,dm_code,credit_use_months,department_amt,g_oversea_amt,g_mailorder_amt,g_department_amt,g_hotel_amt,g_food_amt,g_hospital_amt,g_convini_amt,g_ocap_amt,g_other_amt,gaibu_amt,department_cnt,g_oversea_cnt,g_mailorder_cnt,g_department_cnt,g_hotel_cnt,g_food_cnt,g_hospital_cnt,g_convini_cnt,g_ocap_cnt,g_other_cnt,gaibu_cnt,seg,cancel_year,cancel_month,cancel_day
20201,155223064413,2020/07/16,20200731,4980670002258008,外商,Visa,郵送,20190111,0.0,19330305,80歳以上,男性,500.0,06_600万円未満,滋賀県,9.0,93.0,A,0.0,0.0,20130902,6年〜7年未満,0.0,0.0,0.0,201310,0,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-OG,2020,7,16
20202,155223066426,2020/07/27,20200731,4980670003330004,外商,Visa,郵送,20200102,0.0,19450321,75〜79歳,女性,240.0,03_300万円未満,京都府,9.0,93.0,6,0.0,0.0,20130902,6年〜7年未満,0.0,0.0,28.0,201310,0,Ｄ_京都店,1.0,27804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JFR-OG,2020,7,27
20203,155223066475,2020/07/02,20200731,4980670003367006,外商,Visa,郵送,20191220,0.0,19370928,80歳以上,男性,700.0,08_800万円未満,大阪府,0.0,91.0,A,0.0,0.0,20130902,6年〜7年未満,0.0,20.0,169.0,201310,0,Ｄ_京都店,1.0,112822.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JFR-OG,2020,7,2
20204,155223077266,2020/07/06,20200731,4980670009239001,外商,Visa,郵送,20160831,0.0,19250310,80歳以上,女性,500.0,06_600万円未満,京都府,0.0,40.0,A,0.0,0.0,20130902,6年〜7年未満,0.0,0.0,0.0,201310,0,Ｄ_京都店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-OG,2020,7,6
20205,155223080658,2020/08/29,20200731,4980670010888002,外商,Visa,郵送,20200126,0.0,19370325,80歳以上,女性,330.0,04_400万円未満,東京都,9.0,93.0,A,0.0,0.0,20130902,6年〜7年未満,0.0,0.0,28.0,201310,0,Ｄ_東京店,1.0,21182.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JFR-OG,2020,8,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24589,155231767296,2020/07/30,20200731,5279262507897006,外商,Visa,郵送,20191009,0.0,19590605,60〜64歳,女性,250.0,03_300万円未満,山口県,0.0,80.0,D,0.0,0.0,20180904,1年〜2年未満,0.0,0.0,12.0,201810,0,Ｄ_下関店,1.0,11682.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JFR-OG,2020,7,30
24590,155226779736,2020/08/04,20200731,5279262700531006,外商,Visa,郵送,20150820,0.0,19441211,75〜79歳,男性,1000.0,11_1000万円以上,高知県,0.0,70.0,A,0.0,0.0,20150402,5年〜6年未満,0.0,0.0,0.0,201505,0,Ｄ_高知店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-OG,2020,8,4
24591,155227095322,2020/07/10,20200731,5279262702526004,外商,Visa,郵送,20181110,0.0,19391012,80歳以上,女性,557.0,06_600万円未満,高知県,0.0,80.0,A,0.0,0.0,20150528,5年〜6年未満,0.0,0.0,0.0,201507,0,Ｄ_高知店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-OG,2020,7,10
24592,155227112051,2020/07/01,20200731,5279262702580001,外商,Visa,郵送,20180715,0.0,19381106,80歳以上,女性,800.0,09_900万円未満,高知県,0.0,91.0,A,0.0,0.0,20150601,5年〜6年未満,0.0,0.0,0.0,201507,0,Ｄ_高知店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JFR-OG,2020,7,1


In [23]:
print(df_new_gaisho.dtypes)

daihyouno                 int64
cancel_date              object
cp_date                   int32
serialno                 object
card                     object
brand                    object
channel                  object
last_use_date             int32
kzk_f                   float64
birthday                  int32
age                      object
gender                   object
income                  float64
income2                  object
prefecture               object
job                     float64
occupation              float64
residense                object
email_f                 float64
web_f                   float64
kanyu_date                int32
membership_term          object
shopping_amt_3month     float64
shopping_amt_6month     float64
shopping_amt_12month    float64
nenkaihi                  int32
kirikae_date              int32
dm_code                  object
credit_use_months       float64
department_amt          float64
g_oversea_amt           float64
g_mailor

In [30]:
#外商退会　属性
q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,income2
        ,age
        ,prefecture
        ,membership_term
        ,case when cancel_day < 25 then '25日以前' else '25日以降' end as cancel_dt_border
        ,case when shopping_amt_12month <700 then '年間70万円未満' else '年間70万円以上' end as anual_amt70_border
        ,count(daihyouno) as sum_no_of_account
        from 
            df_new_gaisho
        group by
            1,2,3,4,5,6,7,8,9
        """
df_gaisho = sqldf(q, locals())
df_gaisho

,card,cancel_year,cancel_month,income2,age,prefecture,membership_term,cancel_dt_border,anual_amt70_border,sum_no_of_account
0,外商,2020,7,00_0円,60〜64歳,東京都,6年〜7年未満,25日以前,年間70万円未満,1
1,外商,2020,7,00_0円,65〜69歳,兵庫県,6年〜7年未満,25日以降,年間70万円未満,1
2,外商,2020,7,00_0円,65〜69歳,奈良県,5年〜6年未満,25日以前,年間70万円未満,1
3,外商,2020,7,00_0円,70〜74歳,岐阜県,6年〜7年未満,25日以前,年間70万円以上,1
4,外商,2020,7,00_0円,70〜74歳,愛知県,6年〜7年未満,25日以前,年間70万円以上,1
...,...,...,...,...,...,...,...,...,...,...
772,外商,2020,8,11_1000万円以上,80歳以上,神奈川県,5年〜6年未満,25日以前,年間70万円未満,1
773,外商,2020,8,11_1000万円以上,80歳以上,神奈川県,6年〜7年未満,25日以前,年間70万円未満,1
774,外商,2020,8,11_1000万円以上,80歳以上,高知県,4年〜5年未満,25日以前,年間70万円未満,1
775,外商,2020,8,11_1000万円以上,80歳以上,高知県,4年〜5年未満,25日以降,年間70万円未満,1


In [31]:
#CSV出力
df_gaisho.to_csv(data_dir /'cancel_by_gaisho_detail.csv', header=True ,index=None, encoding='shift-jis')


q = """
        SELECT
        card
        ,cancel_year
        ,cancel_month
        ,count(daihyouno) as sum_no_of_account
        ,avg(department_amt) as ave_department_amt
        ,avg(g_oversea_amt) as ave_g_oversea_amt
        ,avg(g_mailorder_amt) as ave_g_mailorder_amt
        ,avg(g_department_amt) as ave_g_department_amt
        ,avg(g_hotel_amt) as ave_g_hotel_amt
        ,avg(g_food_amt) as ave_g_food_amt
        ,avg(g_hospital_amt) as ave_g_hospital_amt
        ,avg(g_convini_amt) as ave_g_convini_amt
        ,avg(g_ocap_amt) as ave_g_ocap_amt
        ,avg(g_other_amt) as ave_g_other_amt
        ,avg(gaibu_amt) as ave_gaibu_amt
        ,avg(department_cnt) as ave_department_cnt
        ,avg(g_oversea_cnt) as ave_g_oversea_cnt
        ,avg(g_mailorder_cnt) as ave_g_mailorder_cnt
        ,avg(g_department_cnt) as ave_g_department_cnt
        ,avg(g_hotel_cnt) as ave_g_hotel_cnt
        ,avg(g_food_cnt) as ave_g_food_cnt
        ,avg(g_hospital_cnt) as ave_g_hospital_cnt
        ,avg(g_convini_cnt) as ave_g_convini_cnt
        ,avg(g_ocap_cnt) as ave_g_ocap_cnt
        ,avg(g_other_cnt) as ave_g_other_cnt
        ,avg(gaibu_cnt) as ave_gaibu_cnt

        from 
            _df_demo
        group by
            1,2,3
        """
df_credit = sqldf(q, locals())
df_credit

#CSV出力
df_credit.to_csv(data_dir /'解約_cognos_credit.csv', header=True ,index=None, encoding='shift-jis')